# Taller Perceptron

Carlos Garavito,  
Juan Sebastián Cortes. 
***


1. Implementación de un perceptron y su regla de aprendizaje. El perceptron debe aceptar como entrada un vector de longittud arbitraria, i.e. el número de caracteristicas (regresores) puede ser arbitrario.

2. Implementación de un adaline con su regla de aprendizaje. El adaline debe aceptar como entrada un vector de longitud arbitraria, i.e. el número de caracteristicas (regresores) puede ser arbitrario.

3. Implemente un perceptrón y un adaline que pueda realizar las operaciones logicas AND, OR, XOR y XNOR.

4. Al implementar la AND y la OR, si compara los pesos con lso datos de la clase $1$ (cuya salida es $1$, ¿Qué puede concluir?

5. Solucione el problema de las flores de iris utilizando un perceptrón y un adaline. Utilice solamente dos clases. Compare las respuestas.


## Implementación de un perceptron

<center><img src="images/perceptron.png" width="400"></center>

Se ha visto que el perceptron puede ser descrito como se muestra en la figura, y que matematicamente es de la forma

$$z = \boldsymbol\omega^{\text{T}}\mathbf{x},$$ 

tal que, 

$$
\begin{equation}
  y = \varphi(z)=
    \begin{cases}
      1 & \text{if} & z \geq \theta,\\
      -1 & \text{if} & z < \theta.
    \end{cases}       
\end{equation}$$

Además, se ha definido que la regla de aprendizaje se muestra en dos pasos:

1. Inicializar los pesos $\omega$ a 0 o a números aleatorios pequeños.
2. Para cada dato de entrenamiento $\mathbf{x}^{(i)}$ realizar los siguientes pasos:
    * actualizar le valor de la predicción, $\hat{y}$,
    * Actualizar los pesos.

Esto es, 

$$\boldsymbol\omega := \boldsymbol\omega+\Delta\boldsymbol\omega,$$

$$\Delta\boldsymbol\omega = \eta\mathbf{X}^{\text{T}}(\mathbf{y}-\hat{\mathbf{y}}),$$

donde $\omega$ es el vector de pesos; $\eta$ es la constante de aprendizaje; $\mathbf{X}$ es una matrix de dimensiones $m\times N$, donde $m$ es el número de observaciones, y $N$ el número de entradas (regresores, cvaracteristicas, etc).

Así, la implementación en python se muestra a continuación.

In [ ]:
import numpy as np


## Implementación adaline

## Implementación perceptrón y adaline: AND, OR, XOR, XNOR

## Al implementar la AND y la OR, si compara los pesos con lso datos de la clase $1$ (cuya salida es $1$), ¿Qué puede concluir?

## Solución Iris con perceptron y adaline

### Solución Iris con perceptron 

### Solución Iris con adaline

### Comparación de resultados